## Getting Started

In [1]:
from arcgis.gis import GIS
from arcgis.geoenrichment import enrich
from arcgis.geoenrichment import create_report
from arcgis.geometry import Envelope
from arcgis.features.manage_data import generate_tessellation
from arcgis.features import FeatureSet
from getpass import getpass

In [2]:
# User credentials instead of an API key
password=getpass()
gis = GIS(username="azhdanov_ba", password=password)

········


## Build Analysis Areas

In [3]:
# Create the map
inland_empire_map = gis.map('Redlands, CA, USA', zoomlevel=12)
inland_empire_map

MapView(layout=Layout(height='400px', width='100%'))

In [4]:
# Change basemap
inland_empire_map.basemap = 'openstreetmap'

In [5]:
# Create a hex grid for our map
hexagons = generate_tessellation(
    None, 
    bin_size=1, 
    bin_size_unit='SquareMiles', 
    bin_type='HEXAGON', 
    context={"extent":inland_empire_map.extent}
)

In [8]:
len(hexagons.layer.featureSet.features)

176

In [9]:
# Add layer with hex grid
inland_empire_map.add_layer(hexagons)

### Create polygonal study area using the 176 1-sqmi hexagon features as custom (polygon) analysis areas:

In [10]:
geometry_areas = [{"geometry":f["geometry"].copy()} for f in hexagons.layer.featureSet.features]
geometry_areas[:2]

[{'geometry': {'rings': [[[-13062306.6791, 4028583.2673999965],
     [-13062906.5407, 4027535.6636999995],
     [-13064109.4833, 4027533.6686000004],
     [-13064712.784400001, 4028579.2765000015],
     [-13064113.033599999, 4029627.070100002],
     [-13062909.870900001, 4029629.065899998],
     [-13062306.6791, 4028583.2673999965]]]}},
 {'geometry': {'rings': [[[-13060503.5442, 4029632.6744000018],
     [-13061103.6259, 4028585.0713],
     [-13062306.6791, 4028583.2673999965],
     [-13062909.870900001, 4029629.065899998],
     [-13062309.899799999, 4030676.8592000008],
     [-13061106.6265, 4030678.6636999995],
     [-13060503.5442, 4029632.6744000018]]]}}]

In [11]:
for area in geometry_areas:
    area["geometry"]["spatialReference"]={"wkid":3857}

In [12]:
geometry_areas[:2]

[{'geometry': {'rings': [[[-13062306.6791, 4028583.2673999965],
     [-13062906.5407, 4027535.6636999995],
     [-13064109.4833, 4027533.6686000004],
     [-13064712.784400001, 4028579.2765000015],
     [-13064113.033599999, 4029627.070100002],
     [-13062909.870900001, 4029629.065899998],
     [-13062306.6791, 4028583.2673999965]]],
   'spatialReference': {'wkid': 3857}}},
 {'geometry': {'rings': [[[-13060503.5442, 4029632.6744000018],
     [-13061103.6259, 4028585.0713],
     [-13062306.6791, 4028583.2673999965],
     [-13062909.870900001, 4029629.065899998],
     [-13062309.899799999, 4030676.8592000008],
     [-13061106.6265, 4030678.6636999995],
     [-13060503.5442, 4029632.6744000018]]],
   'spatialReference': {'wkid': 3857}}}]

### Enrich all areas (все 6-гранные ячейки)

- **KeyGlobalFacts.TOTPOP** - total population est.
- **AtRisk.SENIORS_CY** - senior populations est.

*(from **Data Drowser**)*

In [13]:
enriched_areas = enrich(
    study_areas=geometry_areas, 
    analysis_variables=["KeyGlobalFacts.TOTPOP","AtRisk.SENIORS_CY"]
)

In [14]:
# We can see TOTPOP and SENIORS_CY
enriched_areas.head()

,ID,OBJECTID,sourceCountry,aggregationMethod,populationToPolygonSizeRating,apportionmentConfidence,HasData,TOTPOP,SENIORS_CY,SHAPE
0,0,1,US,BlockApportionment:US.BlockGroups;PointsLayer:...,2.191,2.576,1,2863,216,"{""rings"": [[[-117.34069735692039, 33.998370806..."
1,1,2,US,BlockApportionment:US.BlockGroups;PointsLayer:...,2.191,2.576,1,1016,90,"{""rings"": [[[-117.3244995205204, 34.0061859207..."
2,2,3,US,BlockApportionment:US.BlockGroups;PointsLayer:...,2.191,2.576,1,32,6,"{""rings"": [[[-117.30827571292721, 33.998408258..."
3,3,4,US,BlockApportionment:US.BlockGroups;PointsLayer:...,2.191,2.576,1,1856,368,"{""rings"": [[[-117.29207489771376, 34.006219097..."
4,4,5,US,BlockApportionment:US.BlockGroups;PointsLayer:...,2.191,2.576,1,32,6,"{""rings"": [[[-117.27585404557786, 33.998437154..."


In [15]:
# Calculating the percentage of the total population
# who are seniors in each analysis area
enriched_areas["SENIORS_PERCENT"] = enriched_areas["SENIORS_CY"]  / enriched_areas["TOTPOP"] * 100

In [16]:
# We can see new column SENIORS_PERCENT
enriched_areas.head()

,ID,OBJECTID,sourceCountry,aggregationMethod,populationToPolygonSizeRating,apportionmentConfidence,HasData,TOTPOP,SENIORS_CY,SHAPE,SENIORS_PERCENT
0,0,1,US,BlockApportionment:US.BlockGroups;PointsLayer:...,2.191,2.576,1,2863,216,"{""rings"": [[[-117.34069735692039, 33.998370806...",7.544534
1,1,2,US,BlockApportionment:US.BlockGroups;PointsLayer:...,2.191,2.576,1,1016,90,"{""rings"": [[[-117.3244995205204, 34.0061859207...",8.858268
2,2,3,US,BlockApportionment:US.BlockGroups;PointsLayer:...,2.191,2.576,1,32,6,"{""rings"": [[[-117.30827571292721, 33.998408258...",18.750000
3,3,4,US,BlockApportionment:US.BlockGroups;PointsLayer:...,2.191,2.576,1,1856,368,"{""rings"": [[[-117.29207489771376, 34.006219097...",19.827586
4,4,5,US,BlockApportionment:US.BlockGroups;PointsLayer:...,2.191,2.576,1,32,6,"{""rings"": [[[-117.27585404557786, 33.998437154...",18.750000


In [17]:
# Filtering the analysis areas with more than 25% seniors
senior_populated_area = enriched_areas[enriched_areas["SENIORS_PERCENT"] > 25]

In [19]:
# See all these filtered areas
senior_populated_area.head()

,ID,OBJECTID,sourceCountry,aggregationMethod,populationToPolygonSizeRating,apportionmentConfidence,HasData,TOTPOP,SENIORS_CY,SHAPE,SENIORS_PERCENT
8,8,9,US,BlockApportionment:US.BlockGroups;PointsLayer:...,2.191,2.576,1,15,4,"{""rings"": [[[-117.21101065877684, 33.998469274...",26.666667
33,33,34,US,BlockApportionment:US.BlockGroups;PointsLayer:...,2.191,2.576,1,545,159,"{""rings"": [[[-117.16237255451121, 34.021856846...",29.174312
34,34,35,US,BlockApportionment:US.BlockGroups;PointsLayer:...,2.191,2.576,1,436,124,"{""rings"": [[[-117.1461605858152, 34.0140578056...",28.440367
35,35,36,US,BlockApportionment:US.BlockGroups;PointsLayer:...,2.191,2.576,1,1442,399,"{""rings"": [[[-117.12994195251805, 34.021847232...",27.669903
36,36,37,US,BlockApportionment:US.BlockGroups;PointsLayer:...,2.191,2.576,1,503,137,"{""rings"": [[[-117.11373295634729, 34.014043913...",27.236581


In [20]:
# Amount of these filtered areas
senior_populated_area.shape[0]

12

In [21]:
# Move all these filtered areas on the map
inland_empire_map2 = gis.map('Redlands, CA, USA', zoomlevel=12)
inland_empire_map2.basemap = 'openstreetmap'
inland_empire_map2.add_layer(FeatureSet.from_dataframe(senior_populated_area.copy()))
inland_empire_map2

MapView(layout=Layout(height='400px', width='100%'))

In [22]:
inland_empire_map3 = gis.map('Redlands, CA, USA', zoomlevel=12)
inland_empire_map3.basemap = 'openstreetmap'
inland_empire_map3.add_layer(FeatureSet.from_dataframe(senior_populated_area.copy()))

# Draw yonger and senior population on the area for each hexagon
# (larger symbol size = greater percentage of seniors)
inland_empire_map3.add_layer(
    FeatureSet.from_dataframe(enriched_areas),
    {'renderer':'ClassedSizeRenderer','field_name':'SENIORS_PERCENT', 'opacity':0.5}
)

inland_empire_map3

c:\users\alexw\desktop\_my_files\jupyter\introduction_with_jupyter_and_arcgis\venv\lib\site-packages\jupyter_client\session.py:716: UserWarning: Message serialization failed with:
Out of range float values are not JSON compliant
Supporting this message is deprecated in jupyter-client 7, please make sure your message is JSON-compliant
  content = self.pack(content)


MapView(layout=Layout(height='400px', width='100%'))

In [23]:
# Let's find out most senior area
most_senior_area = enriched_areas[enriched_areas.SENIORS_PERCENT == enriched_areas.SENIORS_PERCENT.max()]
most_senior_area

,ID,OBJECTID,sourceCountry,aggregationMethod,populationToPolygonSizeRating,apportionmentConfidence,HasData,TOTPOP,SENIORS_CY,SHAPE,SENIORS_PERCENT
53,53,54,US,BlockApportionment:US.BlockGroups;PointsLayer:...,2.191,2.576,1,2185,734,"{""rings"": [[[-117.1948053960044, 34.0374484789...",33.592677


In [24]:
# Draw into the map the most senior area

fill_symbol = {
  "type": "esriSFS",
  "style": "esriSFSSolid",
  "color": [255,0,0,255],
    "outline": {
     "type": "esriSLS",
     "style": "esriSLSSolid",
     "color": [0,0,0,255],
     "width": 1
    }
}

inland_empire_map3.draw(most_senior_area.iloc[0].SHAPE, symbol = fill_symbol)

## Building Report

Let's dive into this area (**most senior area**) more deeper and build infographic report.

In [25]:
most_senior_study_area = {"geometry":most_senior_area.iloc[0].SHAPE}

# Coronavirus impact planning report
report = create_report(
    study_areas=[most_senior_study_area],
    report={"itemid":"ac36af7d86b74071a1cff5dc11f52ae8"}, 
    out_name="COVID Impact Planning Profile.pdf", 
    out_folder="output"
)

In [26]:
# Also create_report() allow you to build a report in html-format
# Let's build another report: health care & insurace 
report2 = create_report(
    study_areas=[most_senior_study_area], 
    report={"itemid":"d29fcd15f66f46edad86499675b0ec03"}, 
    export_format='html', 
    out_name="Health Care and Insurance.html", 
    out_folder="output"
)